In [1]:
#imports
from scipy.spatial import distance_matrix
from random import randrange
import copy
import matplotlib.pyplot as plt
import imageio
import imageio
import random
from time import time
from tqdm import tqdm

In [3]:
#loading datasets 
import numpy as np
kroa={}
krob={}
np_kroa=[]
np_krob=[]
with open('kroa200.txt') as f:
    lines = f.readlines()
    for line in lines[6:-1]:
        x=list(map(int,line.strip().split(' ')))
        #print(x)
        kroa[x[0]]=(x[1],x[2])
        np_kroa.append((x[1],x[2]))
np_kroa=np.array(np_kroa)
with open('krob200.txt') as f:
    lines = f.readlines()
    for line in lines[6:-1]:
        x=list(map(int,line.strip().split(' ')))
        #print(x)
        krob[x[0]]=(x[1],x[2])
        np_krob.append((x[1],x[2]))

np_kroa=np.array(np_kroa)
np_krob=np.array(np_krob)

#calculating distance matrix - https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance_matrix.html
dista=distance_matrix(np_kroa,np_kroa)
distb=distance_matrix(np_krob,np_krob)

In [4]:
#plotting functions
def plotResult(np_dots, cycles):
    plt.clf()
    plt.plot(dataset[:,0][cycles[0][0]], dataset[:,1][cycles[0][0]], "o", markersize=12)
    plt.plot(dataset[:,0][cycles[1][0]], dataset[:,1][cycles[1][0]], "o", markersize=12)
    plt.plot(dataset[:,0][[*cycles[0],cycles[0][0]]], dataset[:,1][[*cycles[0],cycles[0][0]]], "->")
    plt.plot(dataset[:,0][[*cycles[1],cycles[1][1]]], dataset[:,1][[*cycles[1],cycles[1][1]]], "->")
    
def plotPartResult(np_dots, cycles,i):
    plt.plot(np_dots[:,0][cycles[0][0]], np_dots[:,1][cycles[0][0]], "o", markersize=12)
    plt.plot(np_dots[:,0][cycles[1][0]], np_dots[:,1][cycles[1][0]], "o", markersize=12)
    plt.plot(np_dots[:,0][cycles[0]], np_dots[:,1][cycles[0]], "->")
    plt.plot(np_dots[:,0][cycles[1]], np_dots[:,1][cycles[1]], "->")
    plt.plot(np_dots[:,0], np_dots[:,1], ".")
    plt.savefig(f'plots/{i}.png')
    plt.show()    
def calcDistance(currentCycle, dist):
    distance = 0
    for x,y in zip(currentCycle, [*currentCycle[1:], currentCycle[0]]):
        distance += dist[x][y]
    return distance 

In [7]:
# 3rd variable - Generate neighbour solution
#changes edges internally
def change_edges(cycles, edge1, edge2, internally=False): # dummy internally
    cI= 0 if len(np.where(cycles[0] == edge1)[0])!=0 else 1 
#     print(cycles[cI])
    inA=np.where(cycles[cI] == edge1)[0][0]
    inB=np.where(cycles[cI] == edge2)[0][0]
    inA += 1
    inB += 1
    cycle = cycles[cI]
    cycle[inA: inB] = np.flip(cycle[inA: inB])
    return cycles

In [35]:
#changes vertex internally or externaly between cycles
def change_vertices(cycles, a, b, internally=True):
    if internally:
#         print(len(np.where(cycles[0] == a)))
#         print(np.where(cycles[0] == a))
        cI= 0 if len(np.where(cycles[0] == a)[0])!=0 else 1 
        inA=np.where(cycles[cI] == a)[0][0]
        inB=np.where(cycles[cI] == b)[0][0]
        tmp=cycles[cI][inA]
        cycles[cI][inA]=b
        cycles[cI][inB]=tmp
        #print(cycles[0][inB], cycles[0][inA])
    else:
        cI= 0 if len(np.where(cycles[0] == a)[0])!=0 else 1 
        cI2 = 1-cI
        inA=np.where(cycles[cI] == a)[0][0]
        inB=np.where(cycles[cI2] == b)[0][0]
        tmp=cycles[cI][inA]
        cycles[cI][inA]=b
        cycles[cI2][inB]=tmp
        #pass
    #print(cycles)
    return cycles
    #if aIndex in cycles[0]
    #raise NotImplementedError

In [30]:
# 1st variable - Start cycle - with random assignments
def start_cycle_random(dataset_choice="kroa"):
    cycles={}
    cycles[0]=np.random.choice(200,100,replace=False)
    cycles[1]=np.array([i for i in np.arange(0,200,1) if i not in cycles[0]])
    np.random.shuffle(cycles[1])
    return cycles

In [31]:
# 1st variable - Start cycle - with nearest neighbour method
def start_cycle_heuristic(dataset_choice="kroa"):
    results=[]
    min_result=np.inf
    max_result=0
    if dataset_choice=="kroa":
        dataset=np_kroa
        dic=kroa
        dist=copy.deepcopy(dista)
        dist_stable=copy.deepcopy(dista)
    else:
        dataset=np_krob
        dic=krob
        dist=copy.deepcopy(distb)
        dist_stable=copy.deepcopy(distb)


    #zmiana dystansu przekątnej na nieskonczonosc
    np.fill_diagonal(dist, np.inf)

    cycles={}
    cycles[0]=[randrange(200)]
    cycles[1]=[(np.argsort(dist[cycles[0][0]])[-2])]

    for b in range(2):
        dist[:,int(cycles[b][0])]=np.inf

    chosenCycle=1
    for i in range(0,198):
        chosenCycle = int((chosenCycle + 1)%2)
        candidates=[]
        for z,index in enumerate(cycles[chosenCycle]):
            candidates.append((np.argmin(dist[index]),np.min(dist[index]),z))
        candidates=np.array(candidates)
        x=np.argmin(candidates[:,1])
        cycles[chosenCycle]=np.insert(cycles[chosenCycle],int(candidates[x,2]),int(candidates[x,0]))
        dist[:,int(candidates[x,0])]=np.inf
    
    #print(f"cycle[0]: {calcDistance(cycles[0], dist_stable)}")
    #print(f"cycle[1]: {calcDistance(cycles[1], dist_stable)}")
    
    #SAVING RESULTS 
    results.append(calcDistance(cycles[0],dist_stable)+calcDistance(cycles[1],dist_stable))
    if results[-1]>max_result:
        max_result=results[-1]
    if results[-1]<min_result:
        min_result=results[-1]
#         plt.clf()
#         plt.plot(dataset[:,0][cycles[0][0]], dataset[:,1][cycles[0][0]], "o", markersize=12)
#         plt.plot(dataset[:,0][cycles[1][0]], dataset[:,1][cycles[1][0]], "o", markersize=12)
#         plt.plot(dataset[:,0][cycles[0]], dataset[:,1][cycles[0]], "->")
#         plt.plot(dataset[:,0][cycles[1]], dataset[:,1][cycles[1]], "->")
#         plt.savefig(f'best_NN_{dataset_choice}.png')
    return cycles

In [32]:
# 2nd variable - Find more optimal solution
# switch parameter defines how u find neighbour solutions (is it change_vertices or change_edges func)
def greedy_search(dist, np_dots, init_random = False, switch="change_edges"):
    #specify cycles initialization.
    if init_random:
        cycles = start_cycle_random()
    else:
        cycles = start_cycle_heuristic()
        
    #print(cycles)
    
    if switch == "change_vertices":
        internal_change = change_vertices
    elif switch == "change_edges":
        internal_change = change_edges
    else:
        print(f"Unknow mode {switch}. Select change_vertices or change_edges")
        return
    
    
    
    bestDistance = 0
    bestDistance += calcDistance(cycles[0], dist) 
    bestDistance += calcDistance(cycles[1], dist) 
    #print(f"Initial distance: {bestDistance}")
    
    #plotPartResult(np_dots, cycles)
    
    
    done = False
    while not done:
        done = True
        #-------------calculating possible moves
        possible_moves = []
        
        mesh = np.array(np.meshgrid(cycles[0], cycles[1]))
        combinations = mesh.T.reshape(-1, 2)
        for x in combinations:
            possible_moves.append(["external", x])

        mesh = np.array(np.meshgrid(cycles[0], cycles[0]))
        mesh = np.concatenate((mesh,np.array(np.meshgrid(cycles[1], cycles[1]))),axis=0)
        combinations = mesh.T.reshape(-1, 2)
        for x in combinations:
            possible_moves.append(["internal", x])
        
        possible_moves = np.array(possible_moves)
        i = np.random.choice(range(len(possible_moves)),len(possible_moves),replace=False)
        possible_moves = possible_moves[i]
        

        for move, edges in possible_moves:
#             print(move, edges)
            cyclesCopy = copy.deepcopy(cycles)
            edge1 = edges[0]
            edge2 = edges[1]
            
#             print(edge1, edge2)
            if move == "internal":
                cyclesCopy = internal_change(cyclesCopy, edge1, edge2, internally=True)
            else:
                cyclesCopy = change_vertices(cyclesCopy, edge1, edge2, False)
#             plotPartResult(np_dots, cycles)
            currentDistance = 0
            currentDistance += calcDistance(cyclesCopy[0], dist)
            currentDistance += calcDistance(cyclesCopy[1], dist)
            if currentDistance < bestDistance:
                #print(f"New best distance: {currentDistance}, changed {edge1} with {edge2} on {move}")
                cycles = cyclesCopy
                bestDistance = currentDistance
                done = False
                break
    
#     plotPartResult(np_dots, cycles)
#     plt.show()
    return cycles, bestDistance
    

## Genetic

In [160]:
#TestPopulation
testpopulation=[]
dataset=np_kroa
dist=copy.deepcopy(dista)
for i in tqdm(range(0,20)):
        testpopulation.append(greedy_search(dist, dataset, init_random = True, switch="change_edges"))

  5%|▌         | 1/20 [05:54<1:52:08, 354.13s/it]


KeyboardInterrupt: 

In [85]:
# 1st variable - Start cycle - with nearest neighbour method
def rebuild_cycle_heuristic(cycles,dataset_choice="kroa"):
    results=[]
    min_result=np.inf
    max_result=0
    if dataset_choice=="kroa":
        dataset=np_kroa
        dic=kroa
        dist=copy.deepcopy(dista)
        dist_stable=copy.deepcopy(dista)
    else:
        dataset=np_krob
        dic=krob
        dist=copy.deepcopy(distb)
        dist_stable=copy.deepcopy(distb)


    #zmiana dystansu przekątnej na nieskonczonosc
    np.fill_diagonal(dist, np.inf)

#     cycles={}
#     cycles[0]=[randrange(100)]
#     cycles[1]=[(np.argsort(dist[cycles[0][0]])[-2])]

    for b in range(2):
        for d in range(len(cycles[b])):
            dist[:,int(cycles[b][d])]=np.inf

    chosenCycle=1
    taken=len(cycles[0])+len(cycles[1])
    for i in range(0,200-taken):
        chosenCycle = int((chosenCycle + 1)%2)
        candidates=[]
        for z,index in enumerate(cycles[chosenCycle]):
            candidates.append((np.argmin(dist[index]),np.min(dist[index]),z))
        candidates=np.array(candidates)
        x=np.argmin(candidates[:,1])
        cycles[chosenCycle]=np.insert(cycles[chosenCycle],int(candidates[x,2]),int(candidates[x,0]))
        dist[:,int(candidates[x,0])]=np.inf

#     #SAVING RESULTS 
#     results.append(calcDistance(cycles[0],dist_stable)+calcDistance(cycles[1],dist_stable))
#     if results[-1]>max_result:
#         max_result=results[-1]
#     if results[-1]<min_result:
#         min_result=results[-1]

    return cycles

In [159]:
def evo(dataset_choice="kroa", durationSec=10): 
    if dataset_choice=="kroa":
        dataset=np_kroa
        dic=kroa
        dist=copy.deepcopy(dista)
        dist_stable=copy.deepcopy(dista)
    else:
        dataset=np_krob
        dic=krob
        dist=copy.deepcopy(distb)
        dist_stable=copy.deepcopy(distb)    

    bestBestDist = np.inf
    startTime = time()
    
    population=[]
    
#     for i in tqdm(range(0,4)):
#         #Population zawiera dwuelementowe elementy (cycles,bestDist)
#         population.append(greedy_search(dist, dataset, init_random = True, switch="change_edges"))
    population=copy.deepcopy(testpopulation)
    
    
    while (time() - startTime < durationSec):
        div=0
        chosen=np.random.choice(len(population),2,replace=False)
        parent1=population[chosen[0]][0]
        keys1, values1 = zip(*parent1.items())
        values1=np.array(values1)
        values1=values1.ravel()
        parent2=population[chosen[1]][0]
        #print(values1)
        keys2, values2 = zip(*parent2.items())
        values2=np.array(values2)
        values2=values2.ravel()
        child=copy.deepcopy(parent1)
        #print(child)
        for i_in,i in enumerate(values1):
            #print(i)
            for j_in,j in enumerate(values2):
                if i==j:
                    #print("AAA")
                    #print()
#                     print("==")
#                     print(values1[(i_in-1+len(values1))%len(values1)],values1[i_in],values1[(i_in+1+len(values1))%len(values1)])
#                     print(values2[(i_in-1+len(values2))%len(values2)],values2[i_in],values2[(i_in+1+len(values2))%len(values2)])
#                     print("==")
                    if not values1[(i_in-1+len(values1))%len(values1)]==values2[(j_in-1+len(values2)%len(values2))] and not \
                    values1[(i_in+1+len(values1))%len(values1)]==values2[(j_in+1+len(values2))%len(values2)] and div%2==0:
                        div+=1
                        #print("AAAA")
                        if values1[i_in] in child[0]:
                            child[0]=child[0][child[0] != values1[i_in]]#child[0][.remove(values1[i_in])
                        else:
                            child[1]=child[1][child[1] != values1[i_in]]
        new_cycles=rebuild_cycle_heuristic(child)
        #print(new_cycles)
        np_population=np.array(population)
        childDist=calcDistance(child[0],dist)+calcDistance(child[1],dist)
        print(childDist,np.max(np_population[:,1]))
        if childDist<np.max(np_population[:,1]) and childDist not in np_population[:,1]:
            print(population)
            print("AA")
            population[np.argmax(np_population[:,1])]=(child,childDist)
            print(population)
        #print(np.mean(np.array(population)[:,1]))
        #return new_cycles
                       #USUN Z CHILD TEN WIERZCHOŁEK
           #wylosuj 2 rozwiazania z population
           #Ewoluuj - wywal wszystkie krawedzie ktore sa w 1 a nie ma w 2 (wierzcholki bez krawedzi usuwamy)
           #Usun tylko wierzcholki do ktorych nie dochodza zadne krawedzie
           #Odbuduj zgodnie z metodą z ILS2
           #Jezeli nie ma takiego bestDist w population[:,1] i jest krótszy od najdluzszego to zamien go z nim miejscami
           
    
    #print(population)
    
    #return najmniejszy dist z population i te cykle

In [151]:
evo(dataset_choice="kroa", durationSec=100)

37852.75932496475 37978.74114004651
36530.89086486379 37978.74114004651
37783.943318528894 37978.74114004651
37644.90453211102 37978.74114004651
36445.18080653638 37978.74114004651
38050.19213636432 37978.74114004651
38070.147387446064 37978.74114004651
37644.90453211102 37978.74114004651
36710.879492016655 37978.74114004651
36445.18080653638 37978.74114004651
36710.879492016655 37978.74114004651
37852.75932496475 37978.74114004651
37761.353077565196 37978.74114004651
37236.1734186212 37978.74114004651
37644.90453211102 37978.74114004651
36563.32932695506 37978.74114004651
37414.76801768424 37978.74114004651
37842.33734351059 37978.74114004651
36445.18080653638 37978.74114004651
36445.18080653638 37978.74114004651
36652.540886590126 37978.74114004651
37761.353077565196 37978.74114004651
37761.353077565196 37978.74114004651
37644.90453211102 37978.74114004651
36710.879492016655 37978.74114004651
37761.353077565196 37978.74114004651
36710.879492016655 37978.74114004651
36652.540886590126

36445.18080653638 37978.74114004651
37852.75932496475 37978.74114004651
38050.19213636432 37978.74114004651
36563.32932695506 37978.74114004651
37775.47426210686 37978.74114004651
36652.540886590126 37978.74114004651
37852.75932496475 37978.74114004651
36652.540886590126 37978.74114004651
37978.74114004651 37978.74114004651
36652.540886590126 37978.74114004651
38070.04723733431 37978.74114004651
36710.879492016655 37978.74114004651
37783.943318528894 37978.74114004651
37761.353077565196 37978.74114004651
38066.34835698064 37978.74114004651
36445.18080653638 37978.74114004651
36652.540886590126 37978.74114004651
37414.76801768424 37978.74114004651
37978.74114004651 37978.74114004651
36563.32932695506 37978.74114004651
37414.76801768424 37978.74114004651
36652.540886590126 37978.74114004651
37236.1734186212 37978.74114004651
37852.75932496475 37978.74114004651
36652.540886590126 37978.74114004651
36530.89086486379 37978.74114004651
36563.32932695506 37978.74114004651
37236.1734186212 379

37236.1734186212 37978.74114004651
37773.854502413546 37978.74114004651
36530.89086486379 37978.74114004651
37553.49828471147 37978.74114004651
37783.943318528894 37978.74114004651
38066.34835698064 37978.74114004651
36409.57784120823 37978.74114004651
37775.47426210686 37978.74114004651
36652.540886590126 37978.74114004651
37775.47426210686 37978.74114004651
36409.57784120823 37978.74114004651
38050.19213636432 37978.74114004651
37783.943318528894 37978.74114004651
37761.353077565196 37978.74114004651
37236.1734186212 37978.74114004651
37761.353077565196 37978.74114004651
38341.80238709439 37978.74114004651
37978.74114004651 37978.74114004651
37644.90453211102 37978.74114004651
37236.1734186212 37978.74114004651
37852.75932496475 37978.74114004651
37783.943318528894 37978.74114004651
37852.75932496475 37978.74114004651
37852.75932496475 37978.74114004651
38050.19213636432 37978.74114004651
38070.147387446064 37978.74114004651
37761.353077565196 37978.74114004651
36710.879492016655 379

37236.1734186212 37978.74114004651
37236.1734186212 37978.74114004651
36530.89086486379 37978.74114004651
37978.74114004651 37978.74114004651
37773.854502413546 37978.74114004651
38070.04723733431 37978.74114004651
38050.19213636432 37978.74114004651
36710.879492016655 37978.74114004651
36652.540886590126 37978.74114004651
38070.04723733431 37978.74114004651
37644.90453211102 37978.74114004651
37775.47426210686 37978.74114004651
38050.19213636432 37978.74114004651
37775.47426210686 37978.74114004651
37773.854502413546 37978.74114004651
37761.353077565196 37978.74114004651
37775.47426210686 37978.74114004651
37978.74114004651 37978.74114004651
38341.80238709439 37978.74114004651
37236.1734186212 37978.74114004651
36652.540886590126 37978.74114004651
38070.147387446064 37978.74114004651
36652.540886590126 37978.74114004651
36445.18080653638 37978.74114004651
36652.540886590126 37978.74114004651
37414.76801768424 37978.74114004651
36530.89086486379 37978.74114004651
37775.47426210686 3797

36652.540886590126 37978.74114004651
38070.147387446064 37978.74114004651
37775.47426210686 37978.74114004651
36409.57784120823 37978.74114004651
36445.18080653638 37978.74114004651
38066.34835698064 37978.74114004651
38066.34835698064 37978.74114004651
37414.76801768424 37978.74114004651
37236.1734186212 37978.74114004651
37978.74114004651 37978.74114004651
38070.147387446064 37978.74114004651
36530.89086486379 37978.74114004651
37783.943318528894 37978.74114004651
37414.76801768424 37978.74114004651
37978.74114004651 37978.74114004651
37775.47426210686 37978.74114004651
36409.57784120823 37978.74114004651
37852.75932496475 37978.74114004651
38050.19213636432 37978.74114004651
37236.1734186212 37978.74114004651
36530.89086486379 37978.74114004651
37414.76801768424 37978.74114004651
37236.1734186212 37978.74114004651
38066.34835698064 37978.74114004651
37236.1734186212 37978.74114004651
37775.47426210686 37978.74114004651
37783.943318528894 37978.74114004651
37842.33734351059 37978.741

36710.879492016655 37978.74114004651
37414.76801768424 37978.74114004651
38070.04723733431 37978.74114004651
38070.04723733431 37978.74114004651
38070.147387446064 37978.74114004651
37783.943318528894 37978.74114004651
36652.540886590126 37978.74114004651
38341.80238709439 37978.74114004651
36652.540886590126 37978.74114004651
38066.34835698064 37978.74114004651
37852.75932496475 37978.74114004651
37414.76801768424 37978.74114004651
37761.353077565196 37978.74114004651
37783.943318528894 37978.74114004651
36530.89086486379 37978.74114004651
37553.49828471147 37978.74114004651
37775.47426210686 37978.74114004651
36445.18080653638 37978.74114004651
37236.1734186212 37978.74114004651
37414.76801768424 37978.74114004651
37236.1734186212 37978.74114004651
37761.353077565196 37978.74114004651
36563.32932695506 37978.74114004651
36445.18080653638 37978.74114004651
37783.943318528894 37978.74114004651
36710.879492016655 37978.74114004651
37852.75932496475 37978.74114004651
38070.04723733431 37

36530.89086486379 37978.74114004651
38050.19213636432 37978.74114004651
37414.76801768424 37978.74114004651
36409.57784120823 37978.74114004651
36652.540886590126 37978.74114004651
36563.32932695506 37978.74114004651
36563.32932695506 37978.74114004651
37761.353077565196 37978.74114004651
38050.19213636432 37978.74114004651
37761.353077565196 37978.74114004651
36445.18080653638 37978.74114004651
37414.76801768424 37978.74114004651
36530.89086486379 37978.74114004651
37783.943318528894 37978.74114004651
38341.80238709439 37978.74114004651
36710.879492016655 37978.74114004651
37783.943318528894 37978.74114004651
36652.540886590126 37978.74114004651
37761.353077565196 37978.74114004651
36530.89086486379 37978.74114004651
37978.74114004651 37978.74114004651
36445.18080653638 37978.74114004651
37842.33734351059 37978.74114004651
37783.943318528894 37978.74114004651
37775.47426210686 37978.74114004651
36445.18080653638 37978.74114004651
36530.89086486379 37978.74114004651
36445.18080653638 3

KeyboardInterrupt: 

In [56]:
#print(testpopulation)
keys1, values1 = zip(*testpopulation[0][0].items())
values1=np.array(values1)
values1.ravel()

array([ 31,  23, 158, 173, 120,  48,   3, 100, 146, 116, 114,  52,   0,
       159,  19,  63,  66,  30,  46, 156, 108, 154, 182,   7,  16,  24,
       142,  89,  33, 102,  87, 147,  27,  38,  37,  55, 151, 177,  42,
       104,   4, 195,  70, 129,  71, 138,  49, 184, 167, 172, 149,  90,
        94,  93, 181,  75,  69, 143, 101, 163,  22, 168,  29,  67,  34,
         1, 180,  76, 157, 166,   2,  72,  68, 107,  73,  96, 164,  95,
       125,  86,  51,  10,  83, 169,  43, 152,  65, 118,  91,  18, 189,
       178,  36, 137,   8,  81,  77,  25,  60, 135,  59, 192, 127,  40,
        88, 139,  20, 153,  58, 188, 141, 130, 179, 155,  99,  32,  80,
        44, 196, 103, 165,  47, 121, 115, 187,  62, 193,  50,  15, 117,
       123,   6, 198, 183,  28,  17, 109, 171,  45, 110, 131,  84, 144,
       122,  14,  78, 161,  54,  41, 134, 185, 126, 111, 119, 106,  74,
        21, 133, 145,  57, 140, 170, 199,  97, 113, 128,  82,  61,  85,
       136, 132, 175, 112, 194, 124,  53,   5, 186, 150, 160,  7

In [124]:
# print(testpopulation)

In [158]:
print(np.array(testpopulation)[:,1])

[36563.32932695506 36652.540886590126 36710.879492016655 36445.18080653638
 37255.70781072457 37852.75932496475 37553.49828471147 37978.74114004651
 37644.90453211102 37775.47426210686 37414.76801768424 36530.89086486379
 37236.1734186212 37761.353077565196 37842.33734351059 37565.99970955982
 37783.943318528894 37773.854502413546 37767.145915510606
 36409.57784120823]
